In [10]:
import torch
import torchvision
import onnx
import torch.nn as nn
import torch.nn.functional as F
import onnxruntime

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
pthfile = 'net_param_without_pool.pth'
onnxpath = 'test_without_pool.onnx'
print(torch.__version__)
print(onnx.__version__)
print(onnxruntime.__version__)

1.4.0
1.6.0
1.6.0


In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)  # 32-4=28
        self.conv2 = nn.Conv2d(6, 16, 5)  # 28-4=24
        
        self.fc1 = nn.Linear(16*24*24, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        print("after 1 conv:", x.size())  # [1, 6, 28, 28]
        x = F.relu(self.conv2(x)) 
        print("after 2 conv:", x.size())   # [1, 6, 24, 24]
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    # num_flat_features：计算张量x的总特征量（把每个数字都看出是一个特征，即特征总量）
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension(批大小维度)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=9216, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [18]:
x = torch.ones(1,1,32,32)
net = Net()
net.load_state_dict(torch.load('net_param_without_pool.pth'))
net.eval()
prediction = net(x)
print(prediction)

after 1 conv: torch.Size([1, 6, 28, 28])
after 2 conv: torch.Size([1, 16, 24, 24])
tensor([[-0.0633,  0.4135, -0.2653,  0.2503, -0.3490,  0.2363, -0.0144, -0.2845,
         -0.5569,  0.1053]], grad_fn=<AddmmBackward>)


In [19]:
def test():
    batch_size = 1
    model = Net()
    input_shape = (1, 32, 32)
    
    # Initialize model with the pretrained weights
    map_location = 'gpu' if torch.cuda.is_available() else 'cpu'
    loaded_model = torch.load(pthfile, map_location=map_location)
    model.load_state_dict(loaded_model, strict=False)
    
    # set the model to inference mode
    model.eval()
    
    # data type nchw
    x = torch.rand(batch_size, *input_shape)
    input_names = ['input']
    output_names = ['output']
    # # Export the model
    torch.onnx.export(model, x, 
                      onnxpath, 
                      opset_version=11, 
                      keep_initializers_as_inputs=True, 
                      verbose=True)

In [20]:
test()

after 1 conv: torch.Size([1, 6, 28, 28])
after 2 conv: torch.Size([1, 16, 24, 24])
graph(%input.1 : Float(1, 1, 32, 32),
      %conv1.weight : Float(6, 1, 5, 5),
      %conv1.bias : Float(6),
      %conv2.weight : Float(16, 6, 5, 5),
      %conv2.bias : Float(16),
      %fc1.weight : Float(120, 9216),
      %fc1.bias : Float(120),
      %fc2.weight : Float(84, 120),
      %fc2.bias : Float(84),
      %fc3.weight : Float(10, 84),
      %fc3.bias : Float(10)):
  %11 : Float(1, 6, 28, 28) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[0, 0, 0, 0], strides=[1, 1]](%input.1, %conv1.weight, %conv1.bias) # /home/xiang/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/modules/conv.py:342:0
  %12 : Float(1, 6, 28, 28) = onnx::Relu(%11) # /home/xiang/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/functional.py:914:0
  %13 : Float(1, 16, 24, 24) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[0, 0, 0, 0], strides=[1, 1]](%12, %conv2.weight, 

/home/xiang/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:13: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  del sys.path[0]
/home/xiang/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:15: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  from ipykernel import kernelapp as app
